In [286]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import knn

In [287]:
df = pd.read_csv('data/heart_2020_cleaned.csv')
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [288]:
# Se balancea el dataset para que haya la misma cantidad de individuos 
# con HeartDisease positivo y negativo
total = df.count()['HeartDisease']
noHeartDisease = df.loc[df['HeartDisease'] == 'No']
numNoHeartDisease = int(noHeartDisease.count()['HeartDisease'])
numYesHeartDisease = int(df.loc[df['HeartDisease'] == 'Yes'].count()['HeartDisease'])
# numYesHeartDisease
remove = np.random.choice(noHeartDisease.index, numNoHeartDisease - numYesHeartDisease, replace=False)
df = df.drop(remove)




In [289]:
# Se sustituyen valores categóricos por valores numéricos

dfNum = pd.DataFrame()
for col, t in zip(df.columns, df.dtypes):
    pos = 0
    keys = pd.DataFrame()    
    if (t == 'object'):
        key = dict([(k,i) for i,k in enumerate(df.loc[:, col].unique())])
        dfNum[col] = df[col].map(key)
    else:
        dfNum[col] = df[col]

In [290]:
# Se filtran las columnas redundantes
r = 0.0
corrMat = dfNum.corr()
relevantCols = corrMat.loc[(corrMat['HeartDisease'] > r) | (corrMat['HeartDisease'] < -r)].transpose().columns
dfNum = dfNum.loc[:, relevantCols.to_list()]

In [291]:
# Se separan la columna a predecir de los atributos

individuals = pd.DataFrame(dfNum.iloc[:,1:])
individuals

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
5,28.87,0,0,0,6.0,0.0,0,0,0,0,0,0,0,12.0,0,0,0
10,34.30,0,0,0,30.0,0.0,0,1,1,1,1,0,1,15.0,1,0,0
11,28.71,0,0,0,0.0,0.0,1,0,2,1,0,1,2,5.0,0,0,0
35,32.98,0,0,1,10.0,0.0,0,1,0,1,1,1,1,4.0,0,0,1
42,25.06,1,0,0,0.0,0.0,0,0,3,1,1,0,3,7.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319781,37.12,0,0,0,0.0,0.0,1,1,9,3,0,1,2,7.0,0,0,0
319782,31.89,0,0,0,30.0,30.0,0,0,2,3,0,0,0,4.0,0,0,0
319786,33.20,0,0,0,0.0,0.0,1,0,1,3,1,1,2,8.0,1,0,0
319789,22.22,1,0,0,0.0,0.0,1,0,11,3,0,1,4,8.0,0,0,0


In [292]:
target = pd.DataFrame(dfNum['HeartDisease'])
target

,HeartDisease
5,0
10,0
11,1
35,0
42,0
...,...
319781,0
319782,1
319786,0
319789,1


In [293]:
# Normalización
individuals = pd.DataFrame((individuals - individuals.min())/(individuals.max() - individuals.min()))
individuals


,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
5,0.211870,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.478261,0.0,0.0,0.0
10,0.280146,0.0,0.0,0.0,1.000000,0.0,0.0,1.0,0.083333,0.2,0.333333,0.0,0.25,0.608696,1.0,0.0,0.0
11,0.209858,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.166667,0.2,0.000000,1.0,0.50,0.173913,0.0,0.0,0.0
35,0.263548,0.0,0.0,1.0,0.333333,0.0,0.0,1.0,0.000000,0.2,0.333333,1.0,0.25,0.130435,0.0,0.0,1.0
42,0.163963,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.250000,0.2,0.333333,0.0,0.75,0.260870,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319781,0.315604,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.750000,0.6,0.000000,1.0,0.50,0.260870,0.0,0.0,0.0
319782,0.249843,0.0,0.0,0.0,1.000000,1.0,0.0,0.0,0.166667,0.6,0.000000,0.0,0.00,0.130435,0.0,0.0,0.0
319786,0.266315,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.083333,0.6,0.333333,1.0,0.50,0.304348,1.0,0.0,0.0
319789,0.128253,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.916667,0.6,0.000000,1.0,1.00,0.304348,0.0,0.0,0.0


In [294]:
# Se separa el conjunto de datos en entrenamiento y test

xTrain, xTest, yTrain, yTest = train_test_split(individuals, target, test_size=0.3)
classes = target['HeartDisease'].unique()
classes


array([0, 1])

In [295]:
# Se instancia el clasificador y se añaden los datos

classifier = knn.Knn(5)
classifier.fit(xTrain, yTrain)
# classifier.predSingle(yTrain[0])

In [296]:
# classifier.predSingle(xTrain.iloc[1])

In [297]:
predicted = classifier.pred(xTest)
precision, confusionMatrix = classifier.precision(predicted, yTest, classes)
confusionMatrix = pd.DataFrame(confusionMatrix, columns=np.concatenate((np.array(["Class"]), classes)))
print("Fila = predicho, Columna = real")
print("Precisión = " + str(precision))
confusionMatrix


KeyboardInterrupt: 

In [ ]:
cm = confusion_matrix(yTest, predicted)
dfcm = pd.DataFrame(cm)
dfcm

,0,1
0,62,17
1,37,49
